In [1]:
import os
import re
from os.path import join
from io import StringIO
from glob import glob
import shutil

import sympy
import pandas as pd
from sympy import Symbol, sqrt, cos, pi, symbols
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from collections import OrderedDict

import qnet
from qnet.algebra import *

import QDYN

from src.notebook_plots_v1 import plot_bs_decay, display_hamiltonian, display_eq, show_summary_dicke
from src.single_sided_network_v1 import network_slh
from src.dicke_single_model_v2 import write_dicke_single_model, err_dicke_single
from src.doit_actions_v1 import run_traj_prop

from doit.tools import register_doit_as_IPython_magic

In [2]:
qnet.init_printing()

In [3]:
DOIT_CONFIG = {
    'backend': 'json',
    'dep_file': '.doit_db/2017-08-01_mg_test_sprng.json',
}
register_doit_as_IPython_magic()

$
\newcommand{ket}[1]{\vert #1 \rangle}
\newcommand{bra}[1]{\langle #1 \vert}
\newcommand{Op}[1]{\hat{#1}}
$

# How should the error be evaluated?

## Set up runfolders

In [4]:
SOURCE_RF = './data/dicke1_QSL_1traj_xxx/rf_2nodes_T1_50'

In [5]:
DATA_ROOT = './data/dicke1_test_sprng'

In [6]:
params_data_str = r'''
# n_nodes     T  n_trajs      mcwf  seed
        2  1.50   10000       True    12
'''

def read_bool(s):
    """convert string to boolean"""
    true_vals = ['1', 'true']
    false_vals = ['0', 'false']
    s = s.lower()
    if s in true_vals:
        return True
    elif s in false_vals:
        return False
    else:
        raise ValueError("Cannot convert '%s'" % s)

params_df = pd.read_fwf(
        StringIO(params_data_str), comment='#', header=1,
        names=['n_nodes', 'T', 'n_trajs', 'mcwf', 'seed'],
        converters={
            'n_nodes': int, 'T': float, 'n_trajs': int,
            'mcwf': read_bool, 'seed': int})


In [7]:
params_df

,n_nodes,T,n_trajs,mcwf,seed
0,2,1.5,10000,True,12


In [8]:
def runfolder(row):
    int_part, frac_part = ("%.2f" % row['T']).split('.')
    if row['mcwf']:
        label = 'mcwf_%s' % row['seed']
    else:
        label = 'non_herm'
    rf = 'rf_%dnodes_T%s_%s_%s' % (
        row['n_nodes'], int_part, frac_part, label)
    return join(DATA_ROOT, rf)

In [9]:
def copy_optimized_pulses(from_rf, to_rf):
    for file in glob(join(from_rf, 'pulse*.oct.dat')):
        shutil.copy(file, to_rf)

In [10]:
def task_create_runfolder():
    """Create all necessary runfolders for the runs defined in params_df"""
    slh_models = {}
    runfolders = set()
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        if rf in runfolders:
            raise ValueError("Duplicate runfolder %s, row %d" % (rf, ind))
        runfolders.add(rf)
        n_nodes = int(row['n_nodes'])
        try:
            slh = slh_models[n_nodes]
        except KeyError:
            slh = network_slh(
                n_cavity=2, n_nodes=n_nodes, topology='driven_bs_fb')
            slh_models[n_nodes] = slh
        yield {
            'name': str(rf),
            'actions': [
                (write_dicke_single_model, [slh, ], dict(
                    rf=rf, T=row['T'], theta=0, nt=500, seed=row['seed'],
                    kappa=1.0, E0_cycles=2, mcwf=row['mcwf'], non_herm=True,
                    observables='last')),
                (copy_optimized_pulses, [SOURCE_RF, rf])
            ],
            'targets': [join(rf, 'config'), join(rf, 'pulse1.oct.dat')],
            'uptodate': [True, ] # up to date if targets exist
        }


In [11]:
N_PROCS = 1

In [12]:
def task_propagate():
    """Run optimization for every runfolder from params_df"""
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        n_procs = N_PROCS
        if not row['mcwf']:
            n_procs = 1
        yield {
            'name': str(rf),
            'actions': [
                (run_traj_prop, [rf, ], dict(n_trajs=row['n_trajs'], n_procs=n_procs))],
            'file_dep': [join(rf, 'pulse1.oct.dat'),],
            'targets': [join(rf, 'state_final.dat.1'),]}

In [13]:
%doit create_runfolder

.  create_runfolder:./data/dicke1_test_sprng/rf_2nodes_T1_50_mcwf_12


In [15]:
%doit propagate

-- create_runfolder:./data/dicke1_test_sprng/rf_2nodes_T1_50_mcwf_12
.  propagate:./data/dicke1_test_sprng/rf_2nodes_T1_50_mcwf_12


In [17]:
with open("./data/dicke1_test_sprng/rf_2nodes_T1_50_mcwf_12/P_target_combined.dat") as in_fh:
    fid_table = pd.read_fwf(
            in_fh, comment='#', header=1,
            names=['T', 'Fid', 'Var'])


In [18]:
fid_table

,T,Fid,Var
0,1.5,0.999486,0.999486
1,1.5,0.999486,0.999486
2,1.5,0.999486,0.999486
3,1.5,0.999486,0.999486
4,1.5,0.999486,0.999486
5,1.5,0.999486,0.999486
6,1.5,0.999486,0.999486
7,1.5,0.999486,0.999486
8,1.5,0.999486,0.999486
9,1.5,0.999486,0.999486
